In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Efficient Multi-Party Private Set Intersection

### By Malia Kency and John Owens

First, load imports

In [27]:
import protocol
import helpers
import hashes as h
import bloom_filter as bf

Set constant protocol parameters:

   * $NumPlayers$      = Total number of players $P_i$
   * $PlayerInputSize$ = Size of the player's input sets
   * $N_{BF}$          = Length of the Bloom Filter
   * $k$               = Number of hash functions to use
   * $\kappa           = SecParam$ = Security paremater. 40 in the paper's example
   * $N_{Maxones}$     = Maximum number of ones a player is allowed after cut-and-choose
   * $p$               = percentage of total messages to be used for cut-and-choose
   * $a$               = sampling weight of 1s vs. 0s for every $P_i$

Then calculate the rest:
   * $N_{OT}$ = Total number of Random Oblivious Transfers
   * $m_h^1$ = The number of 1's a player chooses
   * $\gamma = gamma$ = Verifies the correct relationship between $p, \kappa, m_h^1$
   * $\gamma^* = gammaStar$ = Verifies the correct relationship between $p, \kappa, N_{OT}$
   * $\varphi$
   * $\pi$
   * $\epsilon$
   * $\lambda$
   

In [39]:
## Parameters:
NumPlayers = 3 
PlayerInputSize = 20 # 10
SecParam = 40
bitLength = 128

# These parameters are meant for illustration and fast execution
# they are not considered secure or optimal
Nmaxones = 80 # 40
p = 0.3 # 0.25 # Fraction of messages to use for Cut and Choose
a = 0.27 # 0.25 # Probability a 1 is chosen by a player

In [40]:
# Initialize the protocol by calculating parameters,
# creating the players, and generating random inputs
# Note: at least 1 shared value is guaranteed
Protocol = protocol.new(NumPlayers, Nmaxones, PlayerInputSize, SecParam, bitLength, p, a)


k = 5
Not = 667
gamma = 0
gammaStar = 1

Simulating players joining protocol. Total: 3
Player 0 created
Player 1 created
Player 2 created


In [41]:
# Perform the random oblivious transfer simulation for P0...Pt
Protocol.perform_RandomOT()
Protocol.print_PlayerROTTable()
Protocol.print_PlayerMessageStats()


Performing Random Oblivious Transfer simulation. 667 transfers in total:
                                                    P0  \
0    (P1, Bit: 0, b't{\xc4$\x00\xca\xee|\tt\x12cCJ\...   
1    (P1, Bit: 0, b'\x03\xa5\x16\x01O_>\xf0\xcd\xe9...   
2    (P1, Bit: 0, b'\x1c\x1aq\xc8\x81&\xce\xeb\xaf&...   
3    (P1, Bit: 0, b'g$y\xfd\xe0W\xe2\x1a\x0b\xe4qv=...   
4    (P1, Bit: 0, b'b%K\x19\x11\x9c\xf7\xeeM0\xe0E\...   
..                                                 ...   
662  (P1, Bit: 0, b'\x8f\xa0\x02<x\x8cB\xf15\x13m\x...   
663  (P1, Bit: 0, b'\xdc\\\xeb\x91\x01\xaf\x0e\x9e\...   
664  (P1, Bit: 0, b'3\xf3\xc5\x01\xc9\x18\x1f8\x04\...   
665  (P1, Bit: 0, b'=\xeeY\xaf\xd2#\xdfC\xf7\x9bC\x...   
666  (P1, Bit: 0, b'\xa6\xdc\xf0\xec\x13\xbdN|:\x8b...   

                                                    P0  \
0    (P1, Bit: 1, b'\xdd\x0ea\x1fO\x9b\xcd\xb3\xff\...   
1    (P1, Bit: 1, b'\xef\xfdB\xa6\x8b\xecQXV;\x0bg\...   
2    (P1, Bit: 1, b'\xa0\x96\xef\x9a\xde-\xe0\xea\x... 

In [42]:
# Perform cut-and-choose simulation for P0...Pt
Protocol.perform_CutandChoose()


Performing Cut and Choose simulation. Size of c: 200. Size of j: 467


In [43]:
# Create bloom filters for P1...Pt
Protocol.create_BloomFilters()


Creating Bloom Filters. BF length: 124


In [44]:
# Create P1...Pt's injective functions
Protocol.create_InjectiveFunctions()


Creating injective functions for every Pi:
Player 1 injective function valid
Player 2 injective function valid


In [51]:
# Instantiate P0's and P1's rGBF objects
Protocol.create_RandomizedGBFs()


Creating randomized GBF for every Pi:


In [47]:
# P0 performs XOR summation on its own j_messages[injective_func] where bit=1
# P1 performs XOR summation on all P1...Pt's j_messages[injective_func] where bit = P1...Pt's choice
Protocol.perform_XORsummation()

In [48]:
# P0 calculates summary values for all elements of its input set
# P1 calculates summary values for all elements of its input set (Every P1...Pt input values)
Protocol.perform_SummaryValues()

In [49]:
# P1 receives P0s summary values, compares them to its own
# Intersections are recorded and output
Protocol.perform_Output()



Player 0's input set: [437, 526, 513, 1985, 337, 675, 167, 876, 719, 957, 701, 246, 934, 811, 836, 123, 950, 762, 336, 485]
Player 1's input set: [592, 299, 248, 141, 300, 109, 576, 537, 676, 267, 571, 575, 1985, 882, 665, 939, 159, 224, 604, 25]
Player 2's input set: [662, 421, 798, 4, 933, 240, 462, 740, 82, 119, 36, 1985, 89, 751, 473, 506, 328, 438, 631, 862]


Player 0's summary values: [7591003..., 4082754..., 7093200..., 7592668..., 1005307..., 9483742..., 1061967..., 4197387..., 3132539..., 7688586..., 8633950..., 1638326..., 7611480..., 2344013..., 1752146..., 6215280..., 1148473..., 1033882..., 5233896..., 7682777..., ]
Player 1's summary values: [5561615..., 9681150..., 4196136..., 8724530..., 5568165..., 6696660..., 5262687..., 1155814..., 7134999..., 3094897..., 7308618..., 1224593..., 7592668..., 1825128..., 7680404..., 7838787..., 9783627..., 3523488..., 6032161..., 3477550..., ]

Intersections found at these values: [1985]
